# ハイパーパラメータの選択

## ライブラリのインストール及びインポート

### ライブラリのインストール
 `Google Colaboratory`にデフォルトでインストールされていないライブラリのインストールを行います。

In [ ]:
!pip install scikit-optimize
!pip install scikeras

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 1.8 MB/s eta 0:00:00


### ライブラリのインポート
本演習の実装に必要となるライブラリのインポートを行います。  
本演習では、`tensorflow及`び`keras`を使用してニューラルネットワークの実装を行います。  
また、`scikit-learn`や`scikit-optimizer`を使用してハイパーパラメータの調整の実装を行います。

In [ ]:
import os
import random as rn
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer
from scikeras.wrappers import KerasClassifier

#乱数の固定
SEED = 2023
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = 'true'
os.environ['TF_CUDNN_DETERMINISTIC'] = 'true'
np.random.seed(SEED)
rn.seed(SEED)
tf.random.set_seed(SEED)

#ログ表示の指定
os.environ['TF_CPP_MIN_LOG_LEVEL']='3'

## データセットの読み込み及び分割

### データセットの読み込み
本演習では、`scilkit-learn`から提供されている、乳癌診断のデータを使用します。  
特徴量として、デジタル画像から細胞核の特徴を算出したデータ、  
正解ラベルとして乳癌かどうかを表すラベル（0 or 1）が格納されています。

In [ ]:
#scikit-learnが提供している乳がんデータセット
df = pd.read_csv('breast_cancer.csv')
df.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


### データセットの分割
訓練データと評価データの分割を行います。  
本演習では訓練データを8割、評価データを2割とします。

In [ ]:
X = df.drop('target', axis=1)
y = df['target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("訓練データの特徴量:{0} 訓練データの正解ラベル：{1}".format(X_train.shape, y_train.shape))
print("評価データの特徴量:{0} 評価データの正解ラベル：{1}".format(X_test.shape, y_test.shape,))

訓練データの特徴量:(455, 30) 訓練データの正解ラベル：(455,)
評価データの特徴量:(114, 30) 評価データの正解ラベル：(114,)


## モデルの定義

本演習では簡単なニューラルネットワークにより2値分類のモデルを作成します。
モデルの引数は以下の通りです。
- `input_size`：入力層の特徴量の数
- `output_size`：出力層のラベルの数
- `learing_rate`：学習率
- `hidden_layer`：隠れ層の数
- `hidden_unit`：隠れ層のユニットの数

In [ ]:
def build_model(input_size, output_size, learning_rate, hidden_layer, hidden_unit):
    model = tf.keras.Sequential()

    model.add(tf.keras.layers.Input(shape=(input_size,)),)

    for i in range(hidden_layer):
        model.add(tf.keras.layers.Dense(hidden_unit, activation='relu'))

    model.add(tf.keras.layers.Dense(output_size, activation='softmax'))

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

## モデルの学習(ハイパーパラメータの調整無し)

特徴量の数は、`X_train`のカラム数から取得します。  
出力ラベルの数は、2値分類のため2と指定します。  
また、以下のパラメータを指定し学習を行います。  
- 学習率 = 1.0  
- 隠れ層の数 = 8
- 隠れ層のニューロン数 = 8  
- エポック数 = 5  

In [ ]:
input_size = len(X_train.columns)
output_size = 2

normal_model = build_model(input_size, output_size, learning_rate=1.0, hidden_layer=8, hidden_unit=8)
normal_model.fit(X_train, y_train, epochs=5)

accuracy = accuracy_score(y_test, np.argmax(normal_model.predict(X_test), axis=1))
print("Accuracy:{0}".format(accuracy))

Epoch 1/5
15/15 [==============================] - 2s 3ms/step - loss: 2117.8796 - accuracy: 0.5187
Epoch 2/5
15/15 [==============================] - 0s 4ms/step - loss: 125.3651 - accuracy: 0.5670
Epoch 3/5
15/15 [==============================] - 0s 3ms/step - loss: 0.6684 - accuracy: 0.6374
Epoch 4/5
15/15 [==============================] - 0s 3ms/step - loss: 0.6742 - accuracy: 0.5890
Epoch 5/5
4/4 [==============================] - 0s 4ms/step
Accuracy: 0.5877192982456141


学習結果は以下の数値となります。  
 `Accuracy`**: 0.5877192982456141**  
 この数値がハイパーパラメータの調整を行うことで、どのように変わっていくか確認します。

## ハイパーパラメータの調整

### ハイパーパラメータの範囲の設定
`build_model`で指定した3つの引数とエポック数の、合計4つをハイパーパラメータとして調整を行います。  
それぞれの探索パターンは以下の様に指定します。  
- 学習率：**「0.01」「0.1」「1.0」**の3パターン
- 隠れ層の数：**「4」「8」「16」**の3パターン
- ニューロンの数：**「4」「8」「16」**の3パターン  
- エポック数：**「5」「10」「15」**の3パターン

In [ ]:
param_dict = {
    'learning_rate': [0.01, 0.1, 1.0],
    'hidden_layer': [4, 8, 16],
    'hidden_unit': [4, 8, 16],
    'epochs': [5, 10, 15]
    }

#ベースモデルの指定
base_model = KerasClassifier(
    model=build_model,
    verbose=0,
    random_state=0,
    input_size=input_size,
    output_size=output_size,
    learning_rate=param_dict['learning_rate'],
    hidden_layer=param_dict['hidden_layer'],
    hidden_unit=param_dict['hidden_unit'],
    epochs=param_dict['epochs']
    )

### グリッドサーチによる探索
まずは、グリッドサーチによるハイパーパラメータの調整を行います。　  
`grid_search`に、グリッドサーチの設定を定義し実行を行います。  
最良のモデル及びその時のハイパーパラメータを取得し確認しましょう。

In [ ]:
# グリッドサーチ
grid_search = GridSearchCV(estimator=base_model, param_grid=param_dict)
grid_search.fit(X_train, y_train)
grid_best_model = grid_search.best_estimator_
grid_best_params = grid_search.best_params_

#モデルの評価
grid_search_accuracy = accuracy_score(y_test, grid_best_model.predict(X_test))
print("Best Parametor:{0}".format(grid_best_params))
print("Best Model Accuracy:{0}".format(grid_search_accuracy))

Best Parametor: {'epochs': 15, 'hidden_layer': 16, 'hidden_neurons': 16, 'learning_rate': 0.01}
Best Model Accuracy: 0.9473684210526315


 グリッドサーチの結果、以下のハイパーパラメータの値が最良の値となります。  
- `learning_rate`: 0.01  
- `hidden_layer`: 16    
- `hidden_unit`: 16    
- `epochs`: 15  
   
また、学習結果は以下の数値となります。  
  
`Best Model Accuracy` **: 0.9473684210526315**
  
実行時間は**約20分**ほどかかり、非常に長くなっています。

### ランダムサーチによる探索
続いて、ランダムサーチによるハイパーパラメータの調整を行います。　  
`random_search`に、ランダムサーチの設定を定義し実行を行います。   
何回ランダムに実行を行うかは`n_iter`で指定します。  
本演習では**10回**とします。
最良のモデル及びその時のハイパーパラメータを取得し確認しましょう。

In [ ]:
# ランダムサーチ
random_search = RandomizedSearchCV(estimator=base_model, param_distributions=param_dict, n_iter=10, random_state=0)
random_search.fit(X_train, y_train)
random_best_model = random_search.best_estimator_
random_best_params = random_search.best_params_

#モデルの評価
random_search_accuracy = accuracy_score(y_test, random_best_model.predict(X_test))
print("Best Parametor：{0}".format(random_best_params))
print("Best Model Accuracy：{0}".format(random_search_accuracy))

Best Parametor:{'learning_rate': 0.01, 'hidden_unit': 8, 'hidden_layer': 4, 'epochs': 10}
Best Model Accuracy:0.8859649122807017


ランダムサーチの結果、以下のハイパーパラメータの値が最良の値となります。  
- `learning_rate`: 0.01  
- `hidden_layer`: 4    
- `hidden_unit`: 8    
- `epochs`: 10  
  
また、学習結果は以下の数値となります。  
  
`Best Model Accuracy` **: 0.8859649122807017**  
    
実行時間は**約2分**ほどで終わっています。

### ベイズ最適化による探索
続いて、ベイス最適化によるハイパーパラメータの調整を行います。　  
`bayes_search`に、ベイス最適化の設定を定義し実行を行います。   
ランダムサーチと同様に何回調整を行うかは`n_iter`で指定します。  
本演習では**10回**とします。
最良のモデル及びその時のハイパーパラメータを取得し確認しましょう。

In [ ]:
# ベイズ最適化
bayes_search = BayesSearchCV(estimator=base_model, search_spaces=param_dict, n_iter=10, random_state=0)
bayes_search.fit(X_train, y_train)
bayes_best_model = bayes_search.best_estimator_
bayes_best_params = bayes_search.best_params_
bayes_best_score = bayes_search.best_score_

#モデルの評価
bayes_search_accuracy = accuracy_score(y_test, bayes_best_model.predict(X_test))
print("Best Parametor:{0}".format(bayes_best_params))
print("Best Model Accuracy:{0}".format(bayes_search_accuracy))

Best Parametor:OrderedDict([('epochs', 10), ('hidden_layer', 8), ('hidden_unit', 8), ('learning_rate', 0.01)])
Best Model Accuracy:0.9035087719298246


ベイズ最適化の結果、以下のハイパーパラメータの値が最良の値となります。  
- `learning_rate`: 0.01  
- `hidden_layer`: 8   
- `hidden_unit`: 8  
- `epochs`: 10    
  
また、学習結果は以下の数値となります。  
  
`Best Model Accuracy` **: 0.9035087719298246**   
  
実行時間は**約3分**ほどで終わっています。

## まとめ

3つのハイパーパラメータの探索方法を実装しました。  
ここまで、実行してきた結果から分かるように、手法により予測精度や実行時間は異なります。  
実行環境や目的に則した手法を選択しましょう。
     
※テキストの値と実際の実行結果や実行時間は異なる場合もあります。  
   
   
 |探索方法|Accuracy|実行時間|  
 | ---: | :---: | :---: |
 | グリッドサーチ | 0.94 | 20分 |
 | ランダムサーチ | 0.88 | 2分 |
 | ベイズ最適化 | 0.90 | 3分 |  